# Settings import

In [1]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")
print(settings)

Local environment
Root dir: /Users/nico/Documents/GitHub/mlfs-book
Added the following directory to the PYTHONPATH: /Users/nico/Documents/GitHub/mlfs-book
HopsworksSettings initialized!
MLFS_DIR=PosixPath('/Users/nico/Documents/GitHub/mlfs-book/mlfs') HOPSWORKS_API_KEY=SecretStr('**********') HOPSWORKS_PROJECT='mac64' HOPSWORKS_HOST=None AQICN_API_KEY=SecretStr('**********') AQICN_URL='https://api.waqi.info/feed/@9352' AQICN_COUNTRY='italy' AQICN_CITY='rome' AQICN_STREET='arenula' FELDERA_API_KEY=SecretStr('**********') OPENAI_API_KEY=SecretStr('**********') OPENAI_MODEL_ID='gpt-4o-mini' FRAUD_DATA_SIZE=<FraudDatasetSize.SMALL: 'SMALL'> EMBEDDING_MODEL='all-MiniLM-L6-v2' TWO_TOWER_MODEL_EMBEDDING_SIZE=16 TWO_TOWER_MODEL_BATCH_SIZE=2048 TWO_TOWER_NUM_EPOCHS=10 TWO_TOWER_WEIGHT_DECAY=0.001 TWO_TOWER_LEARNING_RATE=0.01 TWO_TOWER_DATASET_VALIDATON_SPLIT_SIZE=0.1 TWO_TOWER_DATASET_TEST_SPLIT_SIZE=0.1 RANKING_DATASET_VALIDATON_SPLIT_SIZE=0.1 RANKING_LEARNING_RATE=0.2 RANKING_ITERATIONS=100 R

In [2]:
import datetime
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [4]:
project = hopsworks.login()

2025-11-17 10:13:01,793 INFO: Initializing external client
2025-11-17 10:13:01,794 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-17 10:13:03,479 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1299605


In [5]:
today = datetime.date.today()

csv_file=f"{root_dir}/data/rome-air-quality-data.csv"

util.check_file_path(csv_file)

# taken from ~/.env. You can also replace settings.AQICN_API_KEY with the api key value as a string "...."
if settings.AQICN_API_KEY is None:
    print("You need to set AQICN_API_KEY either in this cell or in ~/.env")
    sys.exit(1)
print(settings.AQICN_API_KEY)

print(settings.AQICN_URL)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value() 
aqicn_url = settings.AQICN_URL
country = settings.AQICN_COUNTRY
city = settings.AQICN_CITY
street = settings.AQICN_STREET
# If this API call fails (it fails in a github action), then set longitude and latitude explicitly - comment out next line
#latitude, longitude = util.get_city_coordinates(city)
# Uncomment this if API call to get longitude and latitude
latitude = "41.893233"
longitude = "12.475331"


print(f"Found AQICN_API_KEY: {AQICN_API_KEY}")

secrets = hopsworks.get_secrets_api()
# Replace any existing secret with the new value
secret = secrets.get_secret("AQICN_API_KEY")
if secret is not None:
    secret.delete()
    print("Replacing existing AQICN_API_KEY")

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

File successfully found at the path: /Users/nico/Documents/GitHub/mlfs-book/data/rome-air-quality-data.csv
**********
https://api.waqi.info/feed/@9352
Found AQICN_API_KEY: e25df776066fd384b109093084b931096c5b6b08
Replacing existing AQICN_API_KEY
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

In [1]:
# Coordinates fo Rome boundary
min_lat = 41.75
min_long = 12.30
max_lat = 42.10
max_long = 12.75

# Get all stations in the boundary box
url = f"https://api.waqi.info/v2/map/bounds?latlng={min_lat},{min_long},{max_lat},{max_long}&networks=all&token={AQICN_API_KEY}"
data = util.trigger_request(url)
print(data)

stations = data.get("data", [])

for s in stations:
    uid = s["uid"]
    url = f"https://api.waqi.info/feed/@{uid}/?token={AQICN_API_KEY}"
    
    station_data = util.trigger_request(url)
    print(station_data)


NameError: name 'AQICN_API_KEY' is not defined